In [ ]:
is_colab_env = False # Set this to true when running on Google Colab

In [ ]:
if is_colab_env:
    !git clone https://github.com/baltekgajda/LegoSortingRecognition.git

In [ ]:
if is_colab_env:
    # Install newer version of Pytorch
    !pip3 install https://download.pytorch.org/whl/cu100/torch-1.1.0-cp36-cp36m-linux_x86_64.whl
    !pip3 install https://download.pytorch.org/whl/cu100/torchvision-0.3.0-cp36-cp36m-linux_x86_64.whl

In [ ]:
# Import necessary libraries
from __future__ import print_function
from __future__ import division
from google.colab import drive
import torch
import torch.nn as nn
import torch.optim as optim
import utils

from data_loader import load_data
from feature_extraction import initialize_model, train_model
import VGGFactory

In [ ]:
# Setup constants
model_name = 'VGG'
data_dir_local = './data/Base Images'
data_dir_colab = '/content/drive/My Drive/Studia/SNR/lego/'
num_classes = 20
num_of_epochs = 50
input_size = 224

In [ ]:
def setup_data_dir(is_colab_env, data_dir_local, data_dir_colab):
    if is_colab_env:
        drive.mount('/content/drive')
        data_dir = data_dir_colab
    else:
        data_dir = data_dir_local
    
    return data_dir

data_dir = setup_data_dir(is_colab_env, './data/Base Images', '/content/drive/My Drive/Studia/SNR/lego/')

In [ ]:
num_classes = 20
num_of_epochs = 50
input_size = 224

model = VGGFactory.create_model(1, num_classes)
for name, param in model.named_parameters():
    if param.requires_grad:
        print(name)

print()

simple_model = VGGFactory.simplify_model(model)
for name, param in simple_model.named_parameters():
    if param.requires_grad:
        print(name)

print("Initializing Datasets and Dataloaders...")
dataloaders_dict = load_data(data_dir, input_size, 0.3, 0.1, 0.1)

params_to_update = model.parameters()
print("Params to learn:")
params_to_update = []
for name, param in model.named_parameters():
    if param.requires_grad:
        params_to_update.append(param)
        print("\t", name)

optimizer_ft = optim.SGD(params_to_update, lr=0.001, momentum=0.9)

# Setup the loss fxn
criterion = nn.CrossEntropyLoss()

# Detect if we have a GPU available
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

# Necessary to run script on Windows
torch.cuda.current_device()

# Train and evaluate
model_ft, hist = train_model(model, dataloaders_dict, criterion, optimizer_ft, device, num_epochs=num_of_epochs)

# Save trained model
utils.save_model(model_ft, "./models")
